# ⚠️ IMPORTANT: Use the New Tutorial Instead!

## 🎯 A complete working LangGraph tutorial has been created!

**New File**: `langgraph_tutorial_local.ipynb`

This new notebook contains:
- ✅ Complete LangGraph quickstart replication
- ✅ Works with **Ollama + Llama 3.2 3B** (local model)
- ✅ Full tool calling support (no workarounds needed)
- ✅ Step-by-step implementation matching the official tutorial
- ✅ Multiple test examples

## 🚀 Quick Start:

1. **The model is currently downloading** (Llama 3.2 3B - ~2GB)
2. **Open**: `langgraph_tutorial_local.ipynb`
3. **Wait** for the model download to complete
4. **Run** all cells in the new notebook

---

**This notebook below** contains experimental code with Qwen2-0.5B which **doesn't support tool calling**. Keep it for reference, but use the new tutorial for the LangGraph quickstart.

# LangChain Tool Calling with Local Models

## Status: ⚠️ Tool Calling Setup

**Current Issue:** The Qwen2-0.5B model doesn't support native tool calling (function calling).

### ✅ Solution Options:

1. **Ollama + Llama 3.2** (Recommended for local use)
   - Ollama has been installed
   - After restarting VS Code, run in terminal: `ollama pull llama3.2:3b`
   - Then use code: `model = init_chat_model("ollama:llama3.2:3b", temperature=0)`
   
2. **API-based models** (Best tool calling support)
   - OpenAI GPT-4: `model = init_chat_model("openai:gpt-4", temperature=0)`
   - Anthropic Claude: `model = init_chat_model("anthropic:claude-3-5-sonnet", temperature=0)`

3. **Current workaround** (Limited capabilities)
   - Using Qwen2-0.5B with manual tool prompting (see cells below)

In [5]:
from langchain.tools import tool
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.language_models import BaseChatModel
from typing import List, Optional
import json

# For tool calling, we'll use a simpler approach with Qwen2-0.5B
# Note: Small models like Qwen2-0.5B don't have native tool calling,
# so we'll create a simple agent pattern instead

print("Loading Qwen2-0.5B model...")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

# Create a pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1
)

llm = HuggingFacePipeline(pipeline=pipe)

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a / b


tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}

# Create tool descriptions for prompting
tool_descriptions = "\n".join([
    f"- {tool.name}: {tool.description}" for tool in tools
])

print("\nModel loaded! Available tools:")
print(tool_descriptions)
print("\nNote: This model doesn't have native tool calling. For better tool support,")
print("consider using: OpenAI, Anthropic, or installing Ollama with llama3.2 or mistral models.")

Loading Qwen2-0.5B model...


Device set to use cpu



Model loaded! Available tools:
- add: Adds `a` and `b`.

    Args:
        a: First int
        b: Second int
- multiply: Multiply `a` and `b`.

    Args:
        a: First int
        b: Second int
- divide: Divide `a` and `b`.

    Args:
        a: First int
        b: Second int

Note: This model doesn't have native tool calling. For better tool support,
consider using: OpenAI, Anthropic, or installing Ollama with llama3.2 or mistral models.


In [ ]:
# ========================================
# OPTION 2: Proper Tool Calling with Ollama + Llama 3.2
# ========================================
# Run this cell AFTER you've completed these steps:
# 1. Restart VS Code (so Ollama is in PATH)
# 2. Open terminal and run: ollama pull llama3.2:3b
# 3. Then uncomment and run this cell

"""
from langchain.tools import tool
from langchain.chat_models import init_chat_model

# Use Llama 3.2 which supports tool calling
model = init_chat_model(
    "ollama:llama3.2:3b",  # or "ollama:mistral" or "ollama:llama3.1"
    temperature=0
)

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    '''Multiply `a` and `b`.'''
    return a * b

@tool
def add(a: int, b: int) -> int:
    '''Adds `a` and `b`.'''
    return a + b

@tool
def divide(a: int, b: int) -> float:
    '''Divide `a` and `b`.'''
    return a / b

# Bind tools - THIS WILL WORK with llama3.2!
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = model.bind_tools(tools)

# Test it
response = model_with_tools.invoke("What is 25 times 4?")
print("Response:", response)
print("\nTool calls:", response.tool_calls)
"""

## 📋 Summary

### The Problem:
- **Qwen2-0.5B doesn't support tool/function calling** natively
- The `bind_tools()` method only works with models that have built-in function calling

### ✅ Solutions:

| Solution | Tool Support | Setup Required | Performance |
|----------|-------------|----------------|-------------|
| **Qwen2-0.5B (Current)** | ❌ No native support | ✅ Ready now | Fast but limited |
| **Ollama + Llama 3.2** | ✅ Full support | 🔄 Restart VS Code needed | Good (local) |
| **OpenAI/Anthropic** | ✅ Best support | 🔑 API key needed | Excellent (cloud) |

### 🚀 Next Steps to Enable Tool Calling:

1. **Restart VS Code** (so Ollama is available in PATH)
2. Open a new terminal
3. Run: `ollama pull llama3.2:3b`
4. Uncomment and run the cell below

### Models with Tool Calling Support:
- ✅ Llama 3.1/3.2 (via Ollama)
- ✅ Mistral (via Ollama)
- ✅ GPT-4, GPT-3.5-turbo (OpenAI)
- ✅ Claude 3.5, Claude 3 (Anthropic)
- ❌ Qwen2-0.5B (too small)